In [1]:
#translates Steve Yeager's NCL scripts into python w/o using dask
#Processes and bias corrects CESM-DPLE output and writes out as a netcdf4
#Will work for CAM and POP fields
#Should be able to (eventually) handle annual, seasonal, and monthly means
#this version does not use dask b/c throughput is the main issue, so limitation
#comes from I/O, and adding more processors does not speed things up

#I've marked in ALL CAPS places that need to be altered
#-Liz Maroon 9/3/2018

#Updated for xarray v0.11.2, 2/15/19

#import packages
import xarray as xr                   #for netcdf manipulation
import numpy as np                    #for numerics
from collections import OrderedDict   #for setting netcdf attributes
import os                             #these last three packages used to detect username/script location
import pwd
import sys
import glob
import datetime

%matplotlib inline

In [5]:
#SET VARIABLE INFO HERE:
VAR='HMXL' #VAR='TS'
MODEL='OCN'# MODEL='ATM' #SET HERE IF PROCESSING CAM OR POP OUTPUT - can write catches for LND/ICE later as needed
WHICHMEAN='ANN'#'MON'  #For ocean monthly means, use a with-dask script #can also do any mean of consecutive months


#WHERE ARE DPLE FILES CURRENTLY:
DPLE_DIR='/glade/scratch/kristenk/dple/netcdf'

#WHERE AND WHAT DO YOU WANT TO CALL OUTPUT FILES:
RAWDPOUT='/glade/scratch/kristenk/dple_stuff/CESM-DP-LE.'+VAR+'.'+WHICHMEAN.lower()+'mean.nc'
DRIFTOUT='/glade/scratch/kristenk/dple_stuff/CESM-DP-LE.'+VAR+'.'+WHICHMEAN.lower()+'mean.drift.nc'
ANOMOUT='/glade/scratch/kristenk/dple_stuff/CESM-DP-LE.'+VAR+'.'+WHICHMEAN.lower()+'mean.anom.nc'


In [6]:
#Make array for start years
first_syear=1954
last_syear=2015
S=xr.DataArray(np.arange(first_syear+1,last_syear+1.5,1,dtype='int'),dims=['S'],coords={'S':np.arange(first_syear+1,last_syear+1.5,1,dtype='int')},name='S')


In [7]:
#fix for time bounds
if MODEL=='OCN': tbname='time_bound'
elif MODEL=='ATM': tbname='time_bnds'

#function for seasonal means - come back to later
def seamean(dataarray,sea):
    monthord='JFMAMJJASOND'
    #pri
    #mon1=sea[0]
    for m2 in monthord:
        if m1==m2: break
    #mon2=sea[1]
    #if 
monthord='JFMAMJJASOND'
for m2 in monthord:
    if 

In [8]:
%%time

oceancoords=['TLAT','TLONG']
da_list=[]

for year in S.values-1:
    print(year)
    #loadthesefiles=sorted(glob.glob(f"{DPLE_DIR}/monthly/{VAR}/b.e11.BDP.f09_g16.{year}-11.*.nc"))
    loadthesefiles=sorted(glob.glob(f"{DPLE_DIR}/b.e11.BDP.f09_g16.{year}-11.*.nc"))
    ds=xr.open_mfdataset(loadthesefiles,concat_dim="M", chunks={"time": 122})
    ds['time'].values=ds[tbname][0,:,0].values
    
    ds=ds.assign_coords(M=np.arange(1,len(loadthesefiles)+1,1,dtype='int'))
    
    #grab attributes first time through looponly:
    if year==S[0]:
        ncattrs=ds.attrs
        varattrs=ds[VAR].attrs
        dimattrs={}
        for dd in ds.dims:
            dimattrs[dd]=ds[dd].attrs
        for cc in ds.coords:
            dimattrs[cc]=ds[cc].attrs
    
    #fix for ocean coordinate size xarray bug
    if MODEL=='OCN': 
        for cc in oceancoords:
            if len(ds[cc].shape)>2:
                tempc=ds[cc][0,:,:].load()
                tempd=tempc.dims
                ds=ds.drop(cc)
                ds=ds.assign_coords(cc=xr.DataArray(tempc.values,coords=[ds[tt].values for tt in tempd],dims=tempd))
                ds=ds.rename({'cc':cc})
    
    da=ds[VAR]
    
    #removing z_t if present for 2d variable :
    if (MODEL=='OCN') and ('z_t' in da.dims):
        da=da.isel(z_t=0)
        da=da.drop('z_t')
    elif ('z_t_150m' in da.dims):
        da=da.isel(z_t_150m=0)
        da=da.drop('z_t_150m')
     
    #get rid of any remaining unwanted dims/coords - want to have left only dims of time, M, lat, lon
    for dd in da.coords:
        if (dd not in oceancoords) and (dd not in da.dims): da=da.drop(dd)

    #time mean
    if WHICHMEAN=='ANN':
        da=da.groupby('time.year').mean('time').isel(year=slice(1,11))  
        #Really should be weighting by months - not doing this for now so can check against Steve's scripts
        da=da.rename({'year':'L'})  
        da['L'].values=np.arange(1,11,1,dtype='int')
        
    elif WHICHMEAN=='MON':
        da=da.rename({'time':'L'})
        da['L'].values=np.arange(1,123,1,dtype='int')
                
    #add here later for seasonal means
    
    da_list.append(da)
    
#one one geyser core: time = 1m 9s for 5 S-years
#one cheyenne core: time = 50 s for 5 S-years

1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016


OSError: no files to open

In [9]:
%%time

#concatenate list into array by dimension S
array=xr.concat(da_list,dim=S)
#array=xr.DataArray(array,coords={'S':array.S,'M':array.M,'L':array.L,'TLAT':(('nlat','nlon'),array.TLAT.values),'TLONG':(('nlat','nlon'),array.TLONG.values)},dims=array.dims)

array=array.transpose('S','L','M','nlat','nlon')

#fast step

CPU times: user 266 ms, sys: 31.1 ms, total: 297 ms
Wall time: 297 ms


In [10]:
%%time

#preparing to turn the DataArray back into a DataSet (so it can be written out as a netcdf)
array.name=VAR
array.attrs=varattrs
dimattrs['S']=OrderedDict([('long_name','start year')])
dimattrs['L']=OrderedDict([('long_name','lead year')])
dimattrs['M']=OrderedDict([('long_name','ensemble member')])

array.attrs=varattrs
for cc in array.coords:
    array[cc].attrs=dimattrs[cc]

#turning DataArray into DataSet and adding ncattrs

newds=array.to_dataset()

newds.attrs=ncattrs
newds.attrs['script']=os.path.basename(sys.argv[0])
now=datetime.datetime.now()
newds.attrs['history']='created by '+pwd.getpwuid(os.getuid()).pw_name+' on '+str(now)        

#fast step

CPU times: user 1.23 ms, sys: 0 ns, total: 1.23 ms
Wall time: 1.25 ms


In [ ]:
%%time 

newds.load()

#one geyser core time for 5 S-years: 3 min 19s
#one cheyenne core time: 2 min 53s
#one geyser core for all S-years: 24 mins

/glade/work/kristenk/miniconda/envs/analysis/lib/python3.7/site-packages/dask/array/numpy_compat.py:28: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


In [ ]:
%%time

#writes out DP array
newds.to_netcdf(RAWDPOUT,engine='netcdf4')

#if already loaded - fast step

In [ ]:
%%time

#calculates drift

climy0=1964
climy1=2014

#newds['vtime']=newds['S']+0.5+newds['L']-1


In [ ]:
%%time

ensmean=newds[VAR].mean('M')

vtime=newds['S']+0.5+newds['L']-1
vtime.values[~((vtime.values>climy0) & (vtime.values<(climy1+1)))]=np.nan
vtime.values[~np.isnan(vtime.values)]=1

drift=(ensmean*vtime).mean('S')
biascorr=newds[VAR]-drift



In [ ]:
%%time

ds_drift=drift.to_dataset(name='drift')
ds_drift.attrs['climatology']=str(climy0)+"-"+str(climy1)+", computed separately for each lead time"
ds_drift.to_netcdf(DRIFTOUT,engine='netcdf4')

#if not preloaded one geyser core 2 min 26s
#if loaded, FAST

In [ ]:
%%time

ds_anom=biascorr.to_dataset(name='anom')
ds_anom.attrs['climatology']=str(climy0)+"-"+str(climy1)+", computed separately for each lead time"
ds_anom.to_netcdf(ANOMOUT,engine='netcdf4')

#one geyser core 2 mins 26s if not preloaded
#if loaded, fast